In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("sampleApp").getOrCreate()

In [ ]:
select(*cols)
selectExpr(*expr)
alias(alias)
colRegex(colName)
filter(condition)/where(condition)
cube(*cols)
rollup(*cols)
drop(*cols)
dropDuplicates(subset=None)/drop_duplicates(subset=None)
dropna(how='any', thresh=None, subset=None)
fillna(value, subset=None)
withColumn(colName, col)
withColumnRenamed(existing, new)
replace(to_replace, value=<no value>, subset=None)

In [115]:
# create df
df = spark.read.csv('/dnbusr1/sambasivaraot/PySpark/input_data/cruise_ship_info.csv',header=True,inferSchema=True)

In [116]:
df.show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 4 rows



### df.select(*cols)
#### Projects a set of expressions and returns a new DataFrame.

In [5]:
df.select('*').show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 4 rows



In [6]:
df.select('Ship_name','Cruise_line').show(4)

+-----------+-----------+
|  Ship_name|Cruise_line|
+-----------+-----------+
|    Journey|    Azamara|
|      Quest|    Azamara|
|Celebration|   Carnival|
|   Conquest|   Carnival|
+-----------+-----------+
only showing top 4 rows



In [7]:
df.select(df.Ship_name).show(4)

+-----------+
|  Ship_name|
+-----------+
|    Journey|
|      Quest|
|Celebration|
|   Conquest|
+-----------+
only showing top 4 rows



In [24]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [26]:
df.select(df.Ship_name,(df.Age <= 26).alias('age')).show(4)

+-----------+----+
|  Ship_name| age|
+-----------+----+
|    Journey|true|
|      Quest|true|
|Celebration|true|
|   Conquest|true|
+-----------+----+
only showing top 4 rows



### df.selectExpr(*expr)
#### Projects a set of SQL expressions and returns a new DataFrame.

In [29]:
sDF = df.selectExpr("age * 2","abs(age)")

In [32]:
sDF.show(2)

+---------+--------+
|(age * 2)|abs(age)|
+---------+--------+
|       12|       6|
|       12|       6|
+---------+--------+
only showing top 2 rows



### alias(alias)
#### Returns a new DataFrame with an alias set.

In [40]:
from pyspark.sql.functions import *
df1 = df.alias('df_a1')
df2 = df.alias('df_a2')

In [44]:
joinDF = df1.join(df2,col('df_a1.Ship_name')== col('df_a2.Ship_name'),'inner')

In [47]:
joinDF.select('df_a1.Ship_name','df_a1.Age').show(4)

+-----------+---+
|  Ship_name|Age|
+-----------+---+
|    Journey|  6|
|      Quest|  6|
|Celebration| 26|
|   Conquest| 11|
+-----------+---+
only showing top 4 rows



### colRegex(colName)

In [52]:
# https://www.geeksforgeeks.org/write-regular-expressions/
colRegexDF = spark.createDataFrame([("a", 1), ("b", 2), ("c",  3)], ["Col1", "Col2"])
colRegexDF.select(df.colRegex("`(Col1)?+.+`")).show()

+----+
|Col2|
+----+
|   1|
|   2|
|   3|
+----+



### filter(condition)/where(condition)
#### Filters rows using the given condition.

In [54]:
df.filter(df.Age > 3).show()
df.where(df.Age > 3).show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [59]:
df.filter("Age > 3").show(4)
df.where("Age >= 28 ").show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 4 rows

+----------------+----------------+---+-------+----------+------+------+-----------------+----+
|       Ship_name|     Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+------

### cube(*cols) / rollup(*cols)
#### Create a multi-dimensional cube for the current DataFrame using the specified columns, so we can run aggregation on them.

In [90]:
r = df.cube("Ship_name",df.Age)

In [91]:
type(r)

pyspark.sql.group.GroupedData

In [93]:
r.min().show(4)

+---------+----+--------+-----------------+---------------+-----------+-----------+----------------------+---------+
|Ship_name| Age|min(Age)|     min(Tonnage)|min(passengers)|min(length)|min(cabins)|min(passenger_density)|min(crew)|
+---------+----+--------+-----------------+---------------+-----------+-----------+----------------------+---------+
|   Spirit|  15|      15|75.33800000000001|          19.56|       8.79|       9.83|                 38.52|     13.0|
|     Wind|  20|      20|            50.76|          17.48|       7.54|       8.74|                 29.04|     6.14|
|  Ventura|   5|       5|            115.0|          35.74|        9.0|      15.32|                 32.18|     12.2|
|  Saphire|null|       9|            113.0|          26.74|       9.51|      13.37|                 42.26|    12.38|
+---------+----+--------+-----------------+---------------+-----------+-----------+----------------------+---------+
only showing top 4 rows



In [94]:
r.count().show(4)

+---------+----+-----+
|Ship_name| Age|count|
+---------+----+-----+
|   Spirit|  15|    1|
|     Wind|  20|    1|
|  Ventura|   5|    1|
|  Saphire|null|    1|
+---------+----+-----+
only showing top 4 rows



In [95]:
rgd = df.rollup("Ship_name",df.Age)

In [96]:
type(rgd)

pyspark.sql.group.GroupedData

In [97]:
rgd.count().show()

+-------------+----+-----+
|    Ship_name| Age|count|
+-------------+----+-----+
|       Spirit|  15|    1|
|         Wind|  20|    1|
|      Ventura|   5|    1|
|      Saphire|null|    1|
|      Freedom|   6|    1|
|         Star|  27|    1|
|        Aries|  22|    1|
|       Melody|  31|    1|
|QueenVictoria|   6|    1|
|      Elation|null|    1|
|    Atlantica|  13|    1|
|       Magica|   9|    1|
|    Rotterdam|  16|    1|
|         Dawn|null|    2|
|   Brilliance|  11|    1|
|        Glory|null|    1|
|     Explorer|  13|    1|
|     Liberty*|   8|    1|
|     Symphony|  18|    1|
|    Statendam|  20|    1|
+-------------+----+-----+
only showing top 20 rows



### drop(*cols)
#### Returns a new DataFrame that drops the specified column.

In [99]:
df.show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 4 rows



In [101]:
df.drop('Age').show(4)

+-----------+-----------+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+------------------+----------+------+------+-----------------+----+
only showing top 4 rows



### dropDuplicates(subset=None)/drop_duplicates(subset=None)

In [107]:
from pyspark.sql import Row
df = spark.sparkContext.parallelize([Row(name='Alice', age=5, height=80),Row(name='Alice', age=5, height=80),Row(name='Alice', age=10, height=80)]).toDF()

In [118]:
df.show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 4 rows



In [120]:
df.dropDuplicates().show(4)

+---------+-----------+---+-------+----------+------+------+-----------------+-----+
|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density| crew|
+---------+-----------+---+-------+----------+------+------+-----------------+-----+
| Rhapsody|        MSC| 36| 16.852|      9.52|  5.41|  3.83|             17.7| 2.97|
|  Saphire|   Princess|  9|  113.0|     26.74|  9.51| 13.37|            42.26|12.38|
|   Legend|   Carnival| 11|   86.0|     21.24|  9.63| 10.62|            40.49|  9.3|
|MarcoPolo|     Orient| 48|  22.08|      8.26|  5.78|  4.25|            26.73|  3.5|
+---------+-----------+---+-------+----------+------+------+-----------------+-----+
only showing top 4 rows



In [121]:
df.drop_duplicates().show(4)

+---------+-----------+---+-------+----------+------+------+-----------------+-----+
|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density| crew|
+---------+-----------+---+-------+----------+------+------+-----------------+-----+
| Rhapsody|        MSC| 36| 16.852|      9.52|  5.41|  3.83|             17.7| 2.97|
|  Saphire|   Princess|  9|  113.0|     26.74|  9.51| 13.37|            42.26|12.38|
|   Legend|   Carnival| 11|   86.0|     21.24|  9.63| 10.62|            40.49|  9.3|
|MarcoPolo|     Orient| 48|  22.08|      8.26|  5.78|  4.25|            26.73|  3.5|
+---------+-----------+---+-------+----------+------+------+-----------------+-----+
only showing top 4 rows



### dropna(how='any', thresh=None, subset=None)
#### Returns a new DataFrame omitting rows with null values. 
##### DataFrame.dropna() and DataFrameNaFunctions.drop() are aliases of each other.

In [122]:
df.dropna().show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 4 rows



In [124]:
df.na.drop().show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 4 rows



### fillna(value, subset=None)
### Replace null values, alias for na.fill(). 
#### DataFrame.fillna() and DataFrameNaFunctions.fill() are aliases of each other.

In [127]:
df.fillna(50).show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [128]:
df.na.fill(50).show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



### withColumn(colName, col)
#### Returns a new DataFrame by adding a column or replacing the existing column that has the same name.

In [129]:
df.withColumn("shp_name",df.Ship_name).show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|   shp_name|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|    Journey|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      Quest|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|Celebration|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|   Conquest|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------+
only showing top 4 rows



### withColumnRenamed(existing, new)
#### Returns a new DataFrame by renaming an existing column. 

In [130]:
df.show(2)

+---------+-----------+---+------------------+----------+------+------+-----------------+----+
|Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|    Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 2 rows



In [131]:
df.withColumnRenamed("Ship_name","name").show(2)

+-------+-----------+---+------------------+----------+------+------+-----------------+----+
|   name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-------+-----------+---+------------------+----------+------+------+-----------------+----+
|Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|  Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
+-------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 2 rows



### replace()
#### Returns a new DataFrame replacing a value with another value. 
#### DataFrame.replace() and DataFrameNaFunctions.replace() are aliases of each other.

In [134]:
df.replace("Azamara","AZM").show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|        AZM|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|        AZM|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [135]:
df.na.replace("Journey","Jrny").show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|       Jrny|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows

